In [133]:
import os
import re
import json
import azureml
import requests
import argparse
from pathlib import Path
from azureml.core.run import Run
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.image import ContainerImage, Image
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import Webservice, AciWebservice

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.53


In [134]:
ws = Workspace.from_config()
model = ws.models['seer']

In [135]:
seer_env = CondaDependencies()
seer_env.add_pip_package('tensorflow==2.0.0-beta1')
seer_env.add_pip_package('numpy')
seer_env.add_pip_package('pillow')
seer_env.add_pip_package('requests')

In [136]:
scoring_yml = 'seer_scoring.yml'
with open(scoring_yml,'w') as f:
    print('Writing out {}'.format(scoring_yml))
    f.write(seer_env.serialize_to_string())
    print('Done!')

Writing out seer_scoring.yml
Done!


In [137]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2,
                                               memory_gb=4,
                                               description='Transfer learning image classification')

In [138]:
# configure the image
image_config = ContainerImage.image_configuration(execution_script='score.py', 
                                                  runtime='python', 
                                                  conda_file=scoring_yml)


In [ ]:
service_name = 'seer-svc'
if service_name in ws.webservices:
    ws.webservices[service_name].delete()
    
service = Webservice.deploy_from_model(workspace=ws,
                                       name=service_name,
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Running.......................................
Succeeded
Image creation operation finished for image seer-svc:4, operation "Succeeded"
Creating service
Running.......................

In [ ]:
with open('deploy.log','w') as f:
    f.write(service.get_logs())
scoring_uri = service.scoring_uri

In [ ]:
tacos = 'https://lh3.googleusercontent.com/-UT5H8nPflkQ/T4tqueyhb_I/AAAAAAAAGl8/1FP7G__Zuys/s640/Lentil+Tacos+close.jpg'
r = requests.post(scoring_uri, json={'image': tacos})
print(r)
response = r.json()
print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
burrito = 'https://www.exploreveg.org/files/2015/05/sofritas-burrito.jpeg'
r = requests.post(scoring_uri, json={'image': tacos})
print(r)
response = r.json()
print(response)

In [132]:
scoring_uri

'http://decce5fb-6d3a-4c2e-989b-c6ac813df7f9.westus2.azurecontainer.io/score'